In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
gcsfuse --debug_fuse bucket /mnt/gcloud-bucket
Using mount point: /mnt/gcloud-bucket
Opening GCS connection...
Opening bucket...
Mounting file system...
daemonize.Run: readFromProcess: sub-process: mountWithArgs: mountWithConn: Mount: mount: running fusermount: exit status 1

stderr:
fusermount: mountpoint is not empty
fusermount: if you are sure this is safe, use the 'nonempty' mount option

In [0]:
!pip install -q keras

In [6]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 11s 188us/step - loss: 0.2647 - acc: 0.9180 - val_loss: 0.0505 - val_acc: 0.9838
Epoch 2/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0880 - acc: 0.9741 - val_loss: 0.0409 - val_acc: 0.9873
Epoch 3/12
 2432/60000 [>.............................] - ETA: 10s - loss: 0.0836 - acc: 0.9778

60000/60000 [==============================] - 11s 182us/step - loss: 0.0678 - acc: 0.9800 - val_loss: 0.0321 - val_acc: 0.9884
Epoch 4/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0563 - acc: 0.9833 - val_loss: 0.0305 - val_acc: 0.9900
Epoch 5/12
49536/60000 [=======================>......] - ETA: 1s - loss: 0.0472 - acc: 0.9860

60000/60000 [==============================] - 11s 181us/step - loss: 0.0472 - acc: 0.9859 - val_loss: 0.0257 - val_acc: 0.9914
Epoch 6/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0422 - acc: 0.9872 - val_loss: 0.0288 - val_acc: 0.9908
Epoch 7/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0385 - acc: 0.9884 - val_loss: 0.0264 - val_acc: 0.9917
Epoch 8/12
  896/60000 [..............................] - ETA: 10s - loss: 0.0563 - acc: 0.9799

60000/60000 [==============================] - 11s 181us/step - loss: 0.0347 - acc: 0.9892 - val_loss: 0.0248 - val_acc: 0.9923
Epoch 9/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0326 - acc: 0.9898 - val_loss: 0.0272 - val_acc: 0.9916
Epoch 10/12
48640/60000 [=======================>......] - ETA: 1s - loss: 0.0310 - acc: 0.9905

60000/60000 [==============================] - 11s 181us/step - loss: 0.0318 - acc: 0.9902 - val_loss: 0.0256 - val_acc: 0.9912
Epoch 11/12
60000/60000 [==============================] - 11s 179us/step - loss: 0.0290 - acc: 0.9908 - val_loss: 0.0256 - val_acc: 0.9918
Epoch 12/12
60000/60000 [==============================] - 11s 180us/step - loss: 0.0281 - acc: 0.9919 - val_loss: 0.0298 - val_acc: 0.9902
Test loss: 0.02983385220271739
Test accuracy: 0.9902


In [0]:
import keras
